In [1]:
!pip install roboflow

from google.colab.patches import cv2_imshow
from roboflow import Roboflow

import albumentations
import csv
import cv2
import numpy as np
import pandas as pd
import random

rf = Roboflow(api_key="7iSKeb2wji6f2yncPgIQ")
project = rf.workspace().project("capstone_olm_logo_recognition")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.


In [1]:
import albumentations
import cv2
import csv
import numpy as np
import random

# Image and Label Path
img_name = '112_Gq5m3KeQJUFNOCzkltNk9t92ucUgueWt9dVVTxUk_jpg.rf.24dd075073cdcefce324a6468c515818'
image_path = '/Users/nickjohnson/downloads/Capstone_OLM_Logo_Recognition.v4i.yolov8/train/images/'+img_name+'.jpg'
image = cv2.imread(image_path)
img_height, img_width, _ = image.shape

def get_bb_params(img_name):
    bb_output = []
    with open('/Users/nickjohnson/downloads/Capstone_OLM_Logo_Recognition.v4i.yolov8/train/labels/'+img_name+'.txt', 'r') as fd:
        reader = csv.reader(fd)
        for bounding_boxes in reader:
            bounding_boxes = bounding_boxes[0].split(" ")
            bounding_boxes = [float(x) for x in bounding_boxes]
            class_id, center_x, center_y, bbox_width, bbox_height = bounding_boxes

            l = (center_x - bbox_width / 2) * img_width
            r = (center_x + bbox_width / 2) * img_width
            t = (center_y - bbox_height / 2) * img_height
            b = (center_y + bbox_height / 2) * img_height

            bb_output.append([l, t, r, b, class_id])
    return bb_output

def apply_augmentation(transformation, image, bb_params):
    transformed = transformation(image=image, bboxes=bb_params, class_labels=[int(x[-1]) for x in bb_params])
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']

    for box in transformed_bboxes:
        l, t, r, b, class_id = box
        cv2.rectangle(transformed_image, (int(l), int(t)), (int(r), int(b)), (255, 0, 0), 2)
    
    return transformed_image

# Define the augmentation pipeline
rotation_degrees = random.randint(45, 270)
transform = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightnessContrast(p=0.5),
    albumentations.Rotate(limit=(rotation_degrees, rotation_degrees), p=1),
], bbox_params=albumentations.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Load bounding box parameters
bounding_box_parameters = get_bb_params(img_name)

# Apply augmentation
augmented_image = apply_augmentation(transform, image, bounding_box_parameters)

# Display the augmented image with bounding boxes
cv2.imshow('Augmented Image', augmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()